# Setup

In [1]:
import pandas as pd
import numpy as numpy
import os
import json
from pathlib import Path

# Own functions
from utils import query
from utils.helpers import *

## Load api key

In [2]:
keys_path = "keys/api_key.json"
# Check if existing keys
if not os.path.exists(keys_path):
    keys = {}
    while True:
        # In case it does not exist, the following are requested 
        name_key = str(input("Enter name of your API key: "))
        api_key = str(input("Enter your API key token: "))

        keys[name_key] = api_key

        answer = str(input("Other API key? y/n")).lower()

        if answer in ["y", "yes", "n", "no"]:
            Path('keys').mkdir(parents=True, exist_ok=True)
            if answer in ["n", "no"]:
                # Finally save keys
                with open(keys_path, 'w') as file:
                    json.dump(keys, file)
                break
        else:
            while answer not in ["y", "yes", "n", "no"]:
                answer = str(input("Invalid answer\n. Other API key? y/n")).lower()


    
with open(keys_path, 'r') as file:
    keys_data = json.load(file) 

printg("API keys loaded successfully")             

API keys loaded successfully


# Load data

In [5]:
printy("Start search")
dict_data_result = query.searchAPI(keys_data["hubspot"])
printg("Finish search")
print("\t Total elements", dict_data_result["total"])

Start search
Finish search
	 Total elements 6936
